# **Performance Calculation**

In [1]:
import pandas as pd

import sys
sys.path.append('../scripts/')
from excel_to_dict import dir2dict, excel2dict
from gas_turbine_performances import PartLoad, ComputeCoeffs

## **Preparing Input Data**

### **GEK 116403 Performance Data**

In [2]:
gek_dict = dir2dict("../data/processed/MS7001EA/")
print(gek_dict.keys())

partial_load_dict = {"PartLoad": excel2dict("../data/processed/PartialLoad.xlsx")}
print(partial_load_dict.keys())

dict_keys(['dPin', 'dPout', 'Pbar', 'RH', 'T'])
dict_keys(['PartLoad'])


### **Climate Data and Load Assumptions**

* Daily demand is considered variable according with following figure
* The daily demand is shown to have a reduction on weekends, around 5%
* The demand is shown to vary during rainy/snow days, we account for this by reducing around 15% demand during rainy season

<img src="../../img/daily_load.png" alt="Fig" width="1000"/>

In [3]:
ambient_df = pd.read_csv("../data/processed/processed_hourly.csv")
ambient_df["date"] = pd.to_datetime(ambient_df["date"])

# Masking for Load Curve

weekend_mask = ambient_df["date"].dt.weekday.isin(range(5, 7))      # Weekends (-5% BaseLoad)

rain_month_mask = ambient_df["date"].dt.month.isin(range(5, 11))    # Rainy Months (May to Oct) (-15% BaseLoad) 

def hour_mask(row):
    if row["date"].hour in range(0, 6):
        return 20
    
    elif row["date"].hour in range(6, 9):
        return 30
    
    elif row["date"].hour in range(9, 17):
        return 50
    
    elif row["date"].hour in range(17, 21):
        return 100
    
    elif row["date"].hour in range(21, 22):
        return 80
    
    else:
        return 20
    
def df_masking(df, new_column):
    
    df_mask = df.copy()
    
    df_mask[new_column] = df_mask.apply(hour_mask, axis=1)
    
    df_mask[new_column] = df_mask[new_column].mask(
        weekend_mask, df_mask[new_column] * (1.00-0.05)
    )
    
    df_mask[new_column] = df_mask[new_column].mask(
        rain_month_mask, df_mask[new_column] * (1.00-0.15)
    )
    
    return df_mask

ambient_df_masked = df_masking(ambient_df, "PartLoad")
ambient_df_masked = ambient_df_masked.set_index("date")
ambient_df_masked.to_csv("../data/processed/processed_hourly_masked.csv")

### **GE 7E Data**

In [4]:
performances_df = PartLoad(gek_dict, partial_load_dict, ambient_df_masked)
performances_df.head()

,Output,Heat Rate,Heat Consumption,Exhaust Temperature,Exhaust Flow
date,,,,,
2020-01-01 04:00:00+00:00,89640.783238,22578.809057,9.580328e+08,906.086808,634976.532683
2020-01-01 05:00:00+00:00,89612.793359,22578.995373,9.577414e+08,906.087259,634783.328697
2020-01-01 06:00:00+00:00,89556.813601,18158.979667,9.571587e+08,1029.088159,632134.680052
2020-01-01 07:00:00+00:00,89510.163803,18159.229403,9.566731e+08,1029.088909,631813.821675
2020-01-01 08:00:00+00:00,89500.833843,18159.279350,9.565760e+08,1029.089059,631749.650000


In [5]:
performances_df.to_csv("../data/results/performances.csv", index=True)